# Training Deep Neural Networks

We will explore the vanishing and exploding gradients problems, tackle complex tasks when you have little labeled data, various optimizers to speed up training models and a few popular regularization techniques.

## Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

![Vanishing-Gradients-in-DNN.png](images/Vanishing-Gradients-in-DNN.png)


### Weight Initialization : Glorot and He Initialization

In their paper, Glorot and Bengio proposed that for the signal to flow properly, variance of the outputs of each layer must be equal to variance of the input; and we need gradience to have equal variance before and after flowing through a layer in the reverse direction. 

Glorot initialization (when using logistic activation function) :

Normal distribution with mean 0 and variance $\sigma^2 = \frac{1}{fan_{avg}}$

Or a uniform distribution between -r and +r, with r = $\sqrt(\frac{3}{fan_{avg}})$

Here, fan$_{avg}$ = (fan$_{in}$ + fan$_{out}$) / 2

This initialization strategy is called **Xavier initialization** or **Glorot initialization**.

Yann LeCun proposed replacing fan$_{avg}$ with fan$_{in}$ in above equation. This strategy is called **LeCun initialization**.

The initialization strategy for the *ReLU* activation functions (including variants) is sometimes called the **He initialization**.

*Table: Initialization parameters for each type of activation function:*

| Initialization | Activation functions | $\sigma^2(Normal)$ |
|-|-|-|
| Glorot | None, tanh, logistic, softmax | $\frac{1}{fan_{avg}}$ |
|He|ReLU and variants| $\frac{2}{fan_{in}}$ |
|LeCun|SELU| $\frac{1}{fan_{in}}$ |

In [2]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

By default, Keras uses Glorot initialization with a uniform distribution.

In [3]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

If you want He initialization with a uniform distribution but based on fan$_{avg}$ rather than fan$_{in}$, you can use VarianceScaling initializer like this:

In [4]:
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)

### Nonsaturating Activation functions

**ReLU** is considered as the much better activation function compared to others. Unfortunately, it is not perfect. It suffers from a problem known as *dying ReLUs*: during training, some neurons stop outputting anything other than 0, especially if you use a large learning rate.

To solve this problem you may use a variant of ReLU, such as the **Leaky ReLU**. This function is defined as: LeakyReLU$_\alpha$(z) = max($\alpha$z, z). $\alpha$ is a slope of the function for z < 0 and is typically set to 0.01. This small slope ensures that the ReLU will never "die"; they can go into a long coma, but eventually wake up.

In the *randomized leaky ReLU* (RReLU), the $\alpha$ is picked randomly in a given range during training and is fixed to an averarage during testing. It acts as a regularizer (reduces the risk of overfitting).

In the *parametric leaky ReLU* (PReLU), the $\alpha$ is authorized to be learned during training (instead of being a hyperparameter, it becomes a parameter that can be modified during backpropagation). PReLU is reported to strongly outperform ReLU on large image datasets, but on smaller datasets it runs the risk of overfitting.

![Leaky ReLU](images/leaky_relu.png)
<center>Leaky ReLU; like ReLU, but with small slope for -ve values.</center>

In [5]:
[m for m in dir(keras.activations) if not m.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'swish',
 'tanh']

In [6]:
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [7]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_val, X_train = X_train_full[:5000], X_train_full[5000:]
y_val, y_train = y_train_full[:5000], y_train_full[5000:]

In [8]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [10]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7955 - accuracy: 0.7362 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7902
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8200
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5553 - accuracy: 0.8157 - val_loss: 0.5350 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5338 - accuracy: 0.8225 - val_loss: 0.5157 - val_accuracy:

Now let's try PReLU:

In [11]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [12]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [13]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
1719/1719 [==============================] - 6s 4ms/step - loss: 1.3461 - accuracy: 0.6209 - val_loss: 0.9255 - val_accuracy: 0.7186
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.8197 - accuracy: 0.7355 - val_loss: 0.7305 - val_accuracy: 0.7628
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6966 - accuracy: 0.7693 - val_loss: 0.6564 - val_accuracy: 0.7880
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6331 - accuracy: 0.7909 - val_loss: 0.6003 - val_accuracy: 0.8050
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5917 - accuracy: 0.8056 - val_loss: 0.5656 - val_accuracy: 0.8180
Epoch 6/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5618 - accuracy: 0.8135 - val_loss: 0.5406 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5390 - accuracy: 0.8205 - val_loss: 0.5196 - val_accuracy:


**Exponential linear unit (ELU)** outperforms all ReLU variants: training time is reduced and the network performed better on the test set.

ELU activation function: 

 $$ ELU_\alpha(z) =   \left\{
\begin{array}{ll}
      \alpha(exp(z) -1) & if z < 0 \\
      z & if z >= 0 \\
\end{array} 
\right.  $$

![ELU](images/ELU.png)


The main drawback of ELU is it is slower to compute than the ReLU functions (due to exponential function).

Implementing ELU in TensorFlow is trivial, just specify the activation function when building each layer:

In [14]:
keras.layers.Dense(10, activation="elu")

Scaled ELU (**SELU**) activation function:

During training, a neural network composed exclusively of a stack of dense layers using the SELU activation function and LeCun initialization will self-normalize: the output of each layer will tend to preserve the same mean and variance during training, which solves the vanishing/exploding gradients problem.

As a result, this activation function outperforms the other activation functions very significantly for such neural nets. Unfortunately, the self-normalizing property of the SELU activation function is easily broken: you cannot use ℓ1 or ℓ2 regularization, regular dropout, max-norm, skip connections or other non-sequential topologies (so recurrent neural networks won't self-normalize). 

However, in practice it works quite well with sequential CNNs. If you break self-normalization, SELU will not necessarily outperform other activation functions.

![SELU](images/selu.png)

Let's create a neural net for Fashion MNIST with 100 hidden layers, using the SELU activation function:

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [17]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

**Note**: Do not forget to scale the inputs to mean 0 and standard deviation 1:

In [18]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_val_scaled = (X_val - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [19]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_val_scaled, y_val))

Epoch 1/5
1719/1719 [==============================] - 25s 15ms/step - loss: 1.2581 - accuracy: 0.5229 - val_loss: 0.7894 - val_accuracy: 0.7192
Epoch 2/5
1719/1719 [==============================] - 27s 16ms/step - loss: 0.8031 - accuracy: 0.7088 - val_loss: 0.6420 - val_accuracy: 0.7772
Epoch 3/5
1719/1719 [==============================] - 30s 18ms/step - loss: 0.6111 - accuracy: 0.7832 - val_loss: 0.5722 - val_accuracy: 0.7944
Epoch 4/5
1719/1719 [==============================] - 30s 17ms/step - loss: 0.5472 - accuracy: 0.8065 - val_loss: 0.5490 - val_accuracy: 0.8116
Epoch 5/5
1719/1719 [==============================] - 25s 15ms/step - loss: 0.5064 - accuracy: 0.8235 - val_loss: 0.5135 - val_accuracy: 0.8228


In [20]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_val_scaled, y_val))

Epoch 1/5
1719/1719 [==============================] - 24s 14ms/step - loss: 1.7682 - accuracy: 0.2741 - val_loss: 1.3060 - val_accuracy: 0.3864
Epoch 2/5
1719/1719 [==============================] - 30s 17ms/step - loss: 1.2256 - accuracy: 0.4781 - val_loss: 1.1808 - val_accuracy: 0.4822
Epoch 3/5
1719/1719 [==============================] - 35s 20ms/step - loss: 0.9104 - accuracy: 0.6321 - val_loss: 0.8720 - val_accuracy: 0.6402
Epoch 4/5
1719/1719 [==============================] - 27s 16ms/step - loss: 0.7976 - accuracy: 0.6908 - val_loss: 0.7200 - val_accuracy: 0.7324
Epoch 5/5
1719/1719 [==============================] - 24s 14ms/step - loss: 0.7394 - accuracy: 0.7217 - val_loss: 0.7023 - val_accuracy: 0.7286


Not great at all, we suffered from the vanishing/exploding gradients problem.

#### So which activation function should you use?

* In general, SELU > ELU > leaky ReLU (all variants) > ReLU > tanh > logistic.
* If the network's architecture prevents it from self-normalizing, ELU may perform better than SELU.
* For runtime latency, prefer Leaky ReLU.
* If you have spare time and computing power, you can use cross-validation to evaluate RReLU(if overfitting) or PReLU (for huge training set).
* If speed is priority, ReLU is the best choice.

### Batch Normalization

Although weight initialization reduces the vanishing/exploding gradient problems at the beginning of training, it doesn't guarantee it won't come back during training.
As neural network learns, it updates the weights over each epoch during training. So what if one of the weights become drastically large than other weights. Well, this large weight will lead to larger output which will cascade causing instability. This is where BN process comes in.

BN is applied to the layers that we choose to apply on within the network.

The BN technique does the following steps:
1. The operation standardizes and normalizes the input values. 
2. The input values are then transformed through scaling (multiplying with $\gamma$) and shifting (adding $\beta$) operations.

Batch Normalization equations:

1. mean: $ \mu_B = \frac{1}{m_B}\sum_{i=1}^{m_B} x^{(i)} $

2. variance: $\sigma_B^2 = \frac{1}{m_B}\sum_{i=1}^{m_B}(x^{(i)} - \mu_B)^2 $

3. normalize (i.e., step 1) : $ \hat x^{(i)} = \frac{x^{(i)} - \mu_B}{\sqrt(\sigma_B^2 + \epsilon)} $

4. scale and shift (i.e., step 2) $ z^{(i)} = \gamma \otimes \hat x^{(i)} + \beta  $

where,
* $ \mu_B $ is the vector of input **means**, evaluated over the whole mini-batch $B$,
* $\sigma_B $ is the vector of input **standard deviations**,
* $ m_B $ is the number of instances,
* $ \hat x^{(i)} $ is the vector of zero-centered and normalized inputs for instance $i$,
* $ \gamma $ is the **output scale** parameter vector for the layer,
* $\otimes $ represents element-wise multiplication,
* $ \beta $ is the **output shift** parameter vector for the layer,
* $\epsilon $ is tiny number that avoids division by zero (typically $10^{-5}$). This is called smoothing term.
* $ z^{(i)} $ is the output of the BN operation. It is the rescaled and shifted version of the inputs.

In [26]:
# Implementation of batch normalization

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [22]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_211 (Dense)            (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_212 (Dense)            (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_213 (Dense)            (None, 10)               

In [23]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [24]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [25]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.8750 - accuracy: 0.7122 - val_loss: 0.5524 - val_accuracy: 0.8226
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5753 - accuracy: 0.8032 - val_loss: 0.4725 - val_accuracy: 0.8468
Epoch 3/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5189 - accuracy: 0.8206 - val_loss: 0.4375 - val_accuracy: 0.8548
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4827 - accuracy: 0.8324 - val_loss: 0.4152 - val_accuracy: 0.8600
Epoch 5/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4565 - accuracy: 0.8408 - val_loss: 0.3997 - val_accuracy: 0.8634
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4398 - accuracy: 0.8474 - val_loss: 0.3866 - val_accuracy: 0.8694
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4242 - accuracy: 0.8514 - val_loss: 0.3762 - val_accuracy

The authors of the BN paper argued in favor of adding the BN layers before the activation functions, rather than after.

In [28]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [29]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [30]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
1719/1719 [==============================] - 8s 5ms/step - loss: 1.0640 - accuracy: 0.6715 - val_loss: 0.6875 - val_accuracy: 0.7836
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6903 - accuracy: 0.7787 - val_loss: 0.5609 - val_accuracy: 0.8172
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6036 - accuracy: 0.8013 - val_loss: 0.5030 - val_accuracy: 0.8370
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5514 - accuracy: 0.8166 - val_loss: 0.4674 - val_accuracy: 0.8446
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5163 - accuracy: 0.8261 - val_loss: 0.4431 - val_accuracy: 0.8514
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4950 - accuracy: 0.8307 - val_loss: 0.4254 - val_accuracy: 0.8580
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4762 - accuracy: 0.8365 - val_loss: 0.4115 - val_accuracy:

### Gradient Clipping
In this technique, it clips the gradient during backpropagation so that they never exceed some threshold. It is most often used in RNN. For other types of networks, BN is usually sufficient.

All Keras optimizers accept clipnorm or clipvalue arguments:

In [31]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)

In [32]:
optimizer = keras.optimizers.SGD(clipnorm=1.0)

## Reusing Pretrained Layers

You should always try to find an existing neural network that accomplishes a similar task to the one you are trying to tackle, then reuse the lower layers of this network. This technique is called **transfer learning**.

![Reusing pretrained layers](images/reusing_pretrained_layers.jpeg)

Try freezing all the reused layers first (i.e., make their weights non-trainable) then train your model and see how it performs.
Then try unfreezing one or two of the top hidden layers to let backpropagation tweak them and see if performance improves.
If you still cannot get food performance, and you have little training data, try dropping the top hidden layer(s) and freezing all the remaining hidden layers to reuse.

### Transfer Learning in Keras

Let's split the fashion MNIST training set in two:

* X_train_A: all images of all items except for sandals and shirts (classes 5 and 6).<br>
* X_train_B: a much smaller training set of just the first 200 images of sandals or shirts.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). 

In [83]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_val_A, y_val_A), (X_val_B, y_val_B) = split_dataset(X_val, y_val)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

X_train_A.shape, X_val_A.shape, X_test_A.shape, X_train_B.shape

((43986, 28, 28), (4014, 28, 28), (8000, 28, 28), (200, 28, 28))

In [87]:
tf.random.set_seed(42)
np.random.seed(42)

In [88]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [90]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [92]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_val_A, y_val_A))

Epoch 1/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.5926 - accuracy: 0.8104 - val_loss: 0.3896 - val_accuracy: 0.8662
Epoch 2/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.3523 - accuracy: 0.8786 - val_loss: 0.3288 - val_accuracy: 0.8827
Epoch 3/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.3170 - accuracy: 0.8895 - val_loss: 0.3014 - val_accuracy: 0.8986
Epoch 4/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2973 - accuracy: 0.8975 - val_loss: 0.2896 - val_accuracy: 0.9011
Epoch 5/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2835 - accuracy: 0.9022 - val_loss: 0.2775 - val_accuracy: 0.9061
Epoch 6/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2730 - accuracy: 0.9060 - val_loss: 0.2736 - val_accuracy: 0.9066
Epoch 7/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2641 - accuracy: 0.9092 - val_loss: 0.2718 - val_accuracy:

In [93]:
model_A.save("my_model_A.h5")

In [94]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [95]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [96]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_val_B, y_val_B))

Epoch 1/20
7/7 [==============================] - 0s 41ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 14ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 13ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 13ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 12ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 14ms/step - loss: 0.3081 - accuracy: 0.9300 - val_loss: 0.3019 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 12ms/step - loss: 0.2800 - accuracy: 0.9350 - val_loss: 0.2804 - val_accuracy: 0.9422
Epoch 8/20
7/7 [===========

In [97]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 784)               3136      
_________________________________________________________________
dense_217 (Dense)            (None, 300)               235200    
_________________________________________________________________
batch_normalization_7 (Batch (None, 300)               1200      
_________________________________________________________________
activation (Activation)      (None, 300)               0         
_________________________________________________________________
dense_218 (Dense)            (None, 100)               30000     
_________________________________________________________________
batch_normalization_8 (Batch (None, 100)              

In [98]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [99]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [100]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [101]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_val_B, y_val_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_val_B, y_val_B))

Epoch 1/4
7/7 [==============================] - 0s 53ms/step - loss: 0.5767 - accuracy: 0.6550 - val_loss: 0.5816 - val_accuracy: 0.6400
Epoch 2/4
7/7 [==============================] - 0s 15ms/step - loss: 0.5404 - accuracy: 0.6800 - val_loss: 0.5443 - val_accuracy: 0.6826
Epoch 3/4
7/7 [==============================] - 0s 18ms/step - loss: 0.5038 - accuracy: 0.7300 - val_loss: 0.5124 - val_accuracy: 0.7110
Epoch 4/4
7/7 [==============================] - 0s 64ms/step - loss: 0.4725 - accuracy: 0.7450 - val_loss: 0.4839 - val_accuracy: 0.7353
Epoch 1/16
7/7 [==============================] - 0s 49ms/step - loss: 0.3948 - accuracy: 0.8150 - val_loss: 0.3455 - val_accuracy: 0.8661
Epoch 2/16
7/7 [==============================] - 0s 17ms/step - loss: 0.2792 - accuracy: 0.9350 - val_loss: 0.2602 - val_accuracy: 0.9300
Epoch 3/16
7/7 [==============================] - 0s 12ms/step - loss: 0.2081 - accuracy: 0.9650 - val_loss: 0.2111 - val_accuracy: 0.9554
Epoch 4/16
7/7 [===============

In [102]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.1408 - accuracy: 0.9705


[0.1408407837152481, 0.9704999923706055]

In [103]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 4ms/step - loss: 0.0683 - accuracy: 0.9935


[0.06834527105093002, 0.9934999942779541]

In [104]:
(100 - 97.05) / (100 - 99.34)

4.469696969696997

Great! We got quite a bit of transfer: the error rate dropped by a factor of 4!

Transfer learning does not work well with small dense networks. It works best with CNN.